Importing libraries

In [1]:
#introducing libraries
!pip install numpy scipy pandas>1.0 matplotlib lmfit colour

In [2]:
#calling up libraries
from datetime import datetime
import pandas as pd
import numpy as np
import scipy
from scipy import optimize
import scipy.integrate as integrate
from scipy.integrate import quad
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from mpl_toolkits import mplot3d
from datetime import timedelta
from lmfit.models import LorentzianModel
import statistics
from matplotlib.collections import LineCollection
from scipy.signal import argrelmax

Importing data from google drive folder, sorting each into absorbance, spectroscope runs, and general spectroscope data.

In [3]:
#import CSV data from drive
from google.colab import drive
import glob
import os
drive.mount('drive', force_remount=True)
folder = "/content/drive/MyDrive/p3ht-pcbm spectra for upload"
all_files = glob.glob(os.path.join(folder, '*.csv'))
Spec_data = {}
Sorb_data = {}
Irr_data = {}
titles = []


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWge4IHRTkJa-akD1klMtZTa9KH7Pnq9r4EPWFkocIT7FNyomrPR324


TIMEOUT: ignored

In [ ]:


for filename in all_files:
  #Simplify name
  new_name = filename.replace(".csv", "").replace("/content/drive/MyDrive/p3ht-pcbm spectra for upload/","")
  #import data
  df = pd.read_csv(filename, sep=',', header=0)
  #Some cleaning
  df.columns = ['Wavelength [nm]', 'Absorbance']
  df=df.set_index('Wavelength [nm]')
  df=df.sort_index()
  print(df)
  Irr_data[new_name] = df
  #change strings to numbers

  titles.append(new_name)
  Spec_data[new_name] = df


Irr_data






In [ ]:
for name in Irr_data:
  if 'blend' in name:
    x = np.asarray(Irr_data[name].index)
    y = np.asarray(Irr_data[name]['Absorbance'])
    if 'Schwarz' in name:
      plt.plot(x,y, label=name, color='b', linestyle='--')
    elif 'bulk' in name:
      plt.plot(x,y, label=name, color='dimgray', linestyle='-.')
    elif 'solution' in name:
      plt.plot(x,y, label=name, color='b', linestyle='-')
    elif 'film' in name:
      plt.plot(x,y, label=name, color='firebrick', linestyle='-')

plt.legend(bbox_to_anchor=(1.4,1))
plt.xlim(400,800)
plt.ylim(-0.1,1.2)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Normalized Absorbance')
plt.show()

for name in Irr_data:
  if ('P3HT' in name):
    x = np.asarray(Irr_data[name].index)
    y = np.asarray(Irr_data[name]['Absorbance'])
    if 'Clafton' in name:
      plt.plot(x, y, label=name, color='royalblue', linestyle=':')
    elif 'Tsoi' in name:
      plt.plot(x, y, label=name, color='silver', linestyle='-.')
  elif 'p3ht' in name:
    x = np.asarray(Irr_data[name].index)
    y = np.asarray(Irr_data[name]['Absorbance'])
    plt.plot(x, y, label=name, color='black', linestyle='-')


plt.legend(bbox_to_anchor=(1.4,1))
plt.xlim(400, 800)
plt.ylim(-0.1,1.2)
plt.xlabel('Wavelength(nm)')
plt.ylabel('Normalized Absorbance')
plt.show()


In [ ]:
#Normalize
for name in Irr_data:
  df = Irr_data[name]
  df_short = df[df.index.to_series().between(400,800)]
  m_val = df_short.max()
  Irr_data[name] = df/m_val

for name in Irr_data:
  if 'NPs' in name:
    x = np.asarray(Irr_data[name].index)
    y = np.asarray(Irr_data[name]['Absorbance'])
    plt.plot(x, y, label=name)

plt.legend(bbox_to_anchor=(1.4,1))
plt.xlim(330, 800)
plt.ylim(-0.1,1.5)
plt.xlabel('Wavelength(nm)')
plt.ylabel('Normalized Absorbance')
plt.show()



In [ ]:
full_data = full_data[full_data.index <3.5]
data_corrected = pd.DataFrame()
bgrd = full_data['quartz']
for name in full_data.columns[2:]:
  data_corrected[name] = full_data[name] - bgrd
data_corrected.plot()
  

In [ ]:
#https://stackoverflow.com/questions/54890415/scipy-optimize-minimize-using-dataframe
import scipy.optimize as opt
#Add composition spectra in most accurate proportions to produce the blend spectrum observed
def main(df):
    x0 = [0.5,0.5, 0.2]
    res = opt.minimize(fun=obj, x0=np.array(x0), args=(df), method="Nelder-Mead")
    return res

def obj(x, df):
    #maybe use a global variable to get the dataframe or via args
    sumSquares = np.sum((df["Y"] - (x[0]*df["X1"] + x[1]*df["X2"] + x[2]*df["X3"]))**2)
    return sumSquares

def dataset(i, ds):
  df = pd.DataFrame()
  for name in ds.columns:
    if ('Blend' in name) and (i in name):
      df['Y'] = ds[name]
    if ('PBDB' in name) and (i in name):
      df['X1'] = ds[name]
    if ('ITIC' in name) and (i in name):
      df['X2'] = ds[name]
    df['X3'] = full_data['quartz']
  return df   

def graph_check(i, ds):
  df = pd.DataFrame()
  for name in ds.columns:
    if ('Blend' in name) and (i in name):
      df['Original'] = ds[name]
    if ('PBDB' in name) and (i in name):
      df['PBDBT'] = ds[name]
    if ('ITIC' in name) and (i in name):
      df['ITIC'] = ds[name]
  df['quartz'] = full_data['quartz']
  a = main(dataset(i, ds))['x'][0]
  b = main(dataset(i, ds))['x'][1]
  c = main(dataset(i, ds))['x'][2]
  df['Extrap= '+str(np.around(a,2))+'*PBDBT + '+str(np.around(b,2))+'*ITIC + ' + str(np.around(c,2)) + "*quartz"] = a*df['PBDBT'] + b*df['ITIC'] + c*df['quartz']
  df.plot(title=i+' spectral combination', style=[':',':',':', ':', '-.'])
  plt.legend(bbox_to_anchor=(1.05,1))
  print(main(dataset(i, ds))['x'])

new_try = data_corrected[data_corrected.index < 2.5] 
graph_check('Cf', data_corrected)

In [ ]:
#Compare normalized version of all material spectra
mats = ['PBDB-T', 'ITIC', 'quartz']
def normal_plot(data, name):
  m = data.max()
  norm = data/m
  plt.plot(norm)
  return norm

#They look a little different, what if we use average as basis spectra?
def avg_norms(mat, data=full_data):
    frame = pd.DataFrame()
    for name in data.columns:
      if mat in name:
        norm = normal_plot(data[name], name)
        frame = frame.append(norm)
    return frame.mean()
frames = {}
for mat in mats:
  frames[mat] = avg_norms(mat)
frames
def norm_basis(lab,data=full_data, frames=frames):
  basis = pd.DataFrame()
  for frame in frames:
    basis[lab+' '+frame] = frames[frame]
  for name in data.columns:
    if (lab in name) and ('Blend' in name): 
      basis[lab+' Blend'] = data[name]
  return basis

def add_test(bar):
  graph_check(bar, norm_basis(bar))

add_test('CSA')



In [ ]:
#set apart different data sets
titles = ['Chlorobenzene', 'Chloroform', 'o-Xylene', 'CSA']
frames = {}
for i in range(4):
  frames[titles[i]] = full_data.iloc[:, (i+3)::4].astype(float)
  frames[titles[i]] = frames[titles[i]].set_index(indx.astype(float))
frames

Plot same materials with different solvents on a graph

In [ ]:
for name in full_data.columns:
  work_with = full_data[full_data.index < 3.5]
  if "Blend" in name:
    plt.subplot(2,2,1)
    plt.title('Blend')
  elif "PBDB-T" in name:
    plt.subplot(2,2,2)
    plt.title('PBDB-T')
  elif "ITIC" in name:
    plt.subplot(2,2,3)
    plt.title('ITIC')
  plt.plot(np.asarray(work_with.index), np.asarray(work_with[name]), label = name)
  plt.rcParams["figure.figsize"] = (20,10)
  plt.ylim(0,1.6)
  plt.ylabel('Absorbance')
  plt.xlabel('Energy(eV)')
  plt.legend()
plt.show()


In [ ]:
#Removing noisy data
fig = plt.figure()
plt.subplots_adjust(hspace=0.4)
for i in range(4):
  data_to_work_with = frames[titles[i]][frames[titles[i]].index <= 1239.842/350 ]
  
  print(data_to_work_with)
#Normalization based on peak
  data_to_work_with.index = 1239.84198/data_to_work_with.index
  for name in data_to_work_with:
    if int(name[0:2]) <=4:
      val = data_to_work_with[name].max()
    else:
      val = 2*data_to_work_with[name].max()

    data_to_work_with[name] = data_to_work_with[name]/val
  axn = fig.add_subplot(2,2,(i+1))
  data_to_work_with['Hypothetical 1:1 Blend'] = (data_to_work_with.iloc[:,1] + data_to_work_with.iloc[:,2])
  val2 = data_to_work_with['Hypothetical 1:1 Blend'].max()
  data_to_work_with['Hypothetical 1:1 Blend'] = data_to_work_with['Hypothetical 1:1 Blend']/val2
  data_to_work_with.plot(xlabel="Energy (eV)", ylabel='Normalized Absorbance', title=titles[i], style = ['-', '--', '--', 'r-.'], ax=axn, figsize=(16,8))

plt.show()

In [ ]:
clean_data = full_data[full_data.index<3.5]
fig = plt.figure(figsize=(16, 8))
plt.subplots_adjust(hspace=0.4)
print(clean_data)
norm_data = pd.DataFrame()
#normalize all data
for name in clean_data.columns:
  val = clean_data[name].max()
  Ep = clean_data[name].idxmax()
  norm_data[name] = clean_data[name]/val
  if "Blend" in name:
    plt.subplot(2,2,1)
    plt.plot(np.asarray(norm_data.index), np.asarray(norm_data[name]), label=name+"(Peak: "+str(np.around(Ep,2))+" eV)")
    plt.xlabel('Energy(eV)')
    plt.ylabel('Normalized Absorbance')
    plt.title('Blend Films')
  elif "PBDB-T" in name:
    plt.subplot(2,2,2)
    plt.plot(np.asarray(norm_data.index), np.asarray(norm_data[name]), label=name+"(Peak: "+str(np.around(Ep,2))+" eV)")
    plt.xlabel('Energy(eV)')
    plt.ylabel('Normalized Absorbance')
    plt.title('PBDB-T Films')
  elif "ITIC" in name:
    plt.subplot(2,2,3)
    plt.plot(np.asarray(norm_data.index), np.asarray(norm_data[name]),label = name+"(Peak: "+str(np.around(Ep,2))+" eV)")
    plt.xlabel('Energy(eV)')
    plt.ylabel('Normalized Absorbance')
    plt.title('ITIC Films')
  else:
    plt.subplot(2,2,4)
    plt.plot(np.asarray(norm_data.index), np.asarray(norm_data[name]), label=name)
    plt.xlabel('Energy(eV)')
    plt.ylabel('Normalized Absorbance')
    plt.title('Baseline Films')
  print(np.around(norm_data.index[argrelmax(np.asarray(norm_data[name]), axis=0, order=1)], 4).astype(str))
  plt.legend()
plt.show()

In [ ]:
full_data.columns